In [340]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
from sklearn.metrics import roc_curve, auc

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [342]:
# we skip first two row and start from 3/2/18 since directionality start at that time
df1 = pd.read_csv('1.gt_bitcoin.csv', usecols=[2], skiprows=2, names=['1.gt_bitcoin'])
df2 = pd.read_csv('2.gt_Buy_Bitcoin.csv', usecols=[2], skiprows=2, names=['2.gt_Buy_Bitcoin'])
df3 = pd.read_csv('3.gt_Sell_Bitcoin.csv', usecols=[2], skiprows=2, names=['3.gt_Sell_Bitcoin'])
df4 = pd.read_csv('4.gt_ethereum.csv', usecols=[2], skiprows=2, names=['4.gt_ethereum'])
df5 = pd.read_csv('5.gt_Ukraine_war.csv', usecols=[2], skiprows=2, names=['5.gt_Ukraine_war'])
df6 = pd.read_csv('6.gt_covid.csv', usecols=[2], skiprows=2, names=['6.gt_covid'])
df7 = pd.read_csv('7.s&p_twitter.csv', usecols=[2], skiprows=2, names=['7.s&p_twitter'])
df8 = pd.read_csv('8.CBDC_uncertainty.csv', usecols=[2], skiprows=2, names=['8.CBDC_uncertainty'])
df9 = pd.read_csv('9.CBDC_atten.csv', usecols=[2], skiprows=2, names=['9.CBDC_atten'])
df10 = pd.read_csv('10.news_senti.csv', usecols=[2], skiprows=2, names=['10.news_senti'])
df11 = pd.read_csv('11.fear&greed.csv', usecols=[2], skiprows=2, names=['11.fear&greed'])
df12 = pd.read_csv('12.tweets_pos.csv', usecols=[2], skiprows=2, names=['12.tweets_pos'])
df13 = pd.read_csv('13.tweets_neg.csv', usecols=[2], skiprows=2, names=['13.tweets_neg'])
df14 = pd.read_csv('14.twitter_btccrash_pos.csv', usecols=[2], skiprows=2, names=['14.twitter_btccrash_pos'])
df15 = pd.read_csv('15.twitter_btccrash_neg.csv', usecols=[2], skiprows=2, names=['15.twitter_btccrash_neg'])

In [344]:
df_concat = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15], axis=1)
df_remove = df_concat.drop(['2.gt_Buy_Bitcoin', '3.gt_Sell_Bitcoin', '8.CBDC_uncertainty'], axis=1)
# replace "inf" with NaN
df_remove = df_remove.replace('inf', 0)
df_remove = df_remove.replace([np.inf, -np.inf], 0)
df_na = df_remove.fillna(0)
df_na

,1.gt_bitcoin,4.gt_ethereum,5.gt_Ukraine_war,6.gt_covid,7.s&p_twitter,9.CBDC_atten,10.news_senti,11.fear&greed,12.tweets_pos,13.tweets_neg,14.twitter_btccrash_pos,15.twitter_btccrash_neg
0,-0.004854,-0.011494,0.000000,0.0,0.006039,0.000321,-0.095687,0.236842,-0.113559,0.514869,-0.138386,0.120306
1,-0.004878,-0.011628,0.000000,0.0,0.003468,-0.000248,-0.097991,0.191489,0.036830,-0.234322,-0.265998,0.381993
2,-0.004902,-0.011765,0.000000,0.0,0.003456,-0.000248,-0.078837,-0.214286,0.063410,0.100377,0.194764,-0.005493
3,0.019704,-0.011905,0.000000,0.0,0.003445,-0.000248,-0.079216,0.250000,-0.097288,-0.182443,-0.223062,-0.145083
4,0.019324,-0.012048,0.000000,0.0,0.001584,-0.000248,-0.051206,0.072727,-0.214515,0.211336,0.090925,0.367505
...,...,...,...,...,...,...,...,...,...,...,...,...
1760,-0.015873,0.142857,-0.011905,0.0,0.000146,-0.000637,-0.202782,-0.034483,-0.192200,0.144226,-0.319385,-0.168254
1761,-0.016129,0.125000,-0.012048,0.0,0.000146,-0.000638,-0.011448,-0.035714,0.172758,0.098971,-0.287368,0.329540
1762,-0.016393,0.111111,-0.012195,0.0,-0.010970,-0.000638,-0.103914,0.037037,-0.127630,0.026982,-0.067539,0.101123
1763,-0.016667,0.100000,-0.012346,0.0,0.013389,-0.000639,-0.064228,0.000000,0.165572,-0.055837,0.233303,-0.110677


In [345]:
bitcoin = pd.read_csv('bitcoin.csv', usecols=[5], skiprows=2, names=['bitcoin'])
ether = pd.read_csv('ethereum.csv', usecols=[5], skiprows=2, names=['ether'])
BDM_exlarge = pd.read_csv('SP_BDM_exlarge.csv', usecols=[5], skiprows=2, names=['BDM_exlarge'])
BDM = pd.read_csv('SP_BDM.csv', usecols=[5], skiprows=2, names=['BDM'])
bitVolumn = pd.read_csv('bitcoin volumn.csv', usecols=[1], names=['bitcoinVolumn'])
bitVolumn = bitVolumn['bitcoinVolumn'].pct_change()
bitVolumnpct= bitVolumn.drop([0], axis=0)
bitVolumnpct = bitVolumnpct.reset_index(drop=True)
sp500 = pd.read_csv('SP500.csv', usecols=[1], names=['sp500'])
sp500 = sp500['sp500'].pct_change()
sp500pct= sp500.drop([0], axis=0)
sp500pct = sp500pct.reset_index(drop=True)
df_control = pd.concat([df_na,bitVolumnpct,sp500pct], axis=1)

# 1. Bitcoin

In [346]:
bitcoin_concat = pd.concat([bitcoin,df_na], axis=1)
train, test = train_test_split(bitcoin_concat, test_size = 0.3)
x_train = train.iloc[0:,1:13]
y_train = train.iloc[0:,0]
x_test = test.iloc[0:,1:13]
y_test = test.iloc[0:,0]
scaler = StandardScaler()
# scale the data
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [347]:
model = LogisticRegression()
model.fit(x_train,y_train)
predictions = model.predict(x_test)
actuals = np.array(y_test)
logistic_acc = model.score(x_test,actuals)
print(logistic_acc)

0.530188679245283


In [348]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

param_grid = [{'solver': ['newton-cg', 'sag', 'lbfgs'],
                 'penalty': ['l2'],
                 'C': [0.01, 0.001, 0.5, 10]},
                {'solver': ['saga'],
                 'penalty': ['l1', 'l2'],
                 'C': [0.1, 0.01, 0.001, 0.5]}
                ]

# Use Grid Search to find the best hyperparameters
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters and corresponding score
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

# Evaluate the model with the best hyperparameters on the testing set
best_lr = grid_search.best_estimator_
y_pred = best_lr.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Best hyperparameters: {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}
Best score: 0.5659919028340081
Accuracy: 0.5377358490566038


In [349]:
confusion_matrix(y_test, y_pred)

array([[112, 135],
       [110, 173]])

## control

In [350]:
bitcoin_control = pd.concat([bitcoin,df_control], axis=1)
train, test = train_test_split(bitcoin_control, test_size = 0.3)
x_train = train.iloc[0:,1:15]
y_train = train.iloc[0:,0]
x_test = test.iloc[0:,1:15]
y_test = test.iloc[0:,0]
scaler = StandardScaler()
# scale the data
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = LogisticRegression()
model.fit(x_train,y_train)
predictions = model.predict(x_test)
actuals = np.array(y_test)
logistic_acc = model.score(x_test,actuals)
print(logistic_acc)

0.5660377358490566


In [351]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

param_grid = [{'solver': ['newton-cg', 'sag', 'lbfgs'],
                 'penalty': ['l2'],
                 'C': [0.01, 0.001, 0.5, 10]},
                {'solver': ['saga'],
                 'penalty': ['l1', 'l2'],
                 'C': [0.1, 0.01, 0.001, 0.5]}
                ]

# Use Grid Search to find the best hyperparameters
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters and corresponding score
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

# Evaluate the model with the best hyperparameters on the testing set
best_lr = grid_search.best_estimator_
y_pred = best_lr.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Best hyperparameters: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
Best score: 0.5862348178137651
Accuracy: 0.5679245283018868


In [352]:
confusion_matrix(y_test, y_pred)

array([[131, 116],
       [113, 170]])

# 2. Ether

In [353]:
ether_concat = pd.concat([ether,df_na], axis=1)
train, test = train_test_split(ether_concat, test_size = 0.3)
x_train = train.iloc[0:,1:13]
y_train = train.iloc[0:,0]
x_test = test.iloc[0:,1:13]
y_test = test.iloc[0:,0]
scaler = StandardScaler()
# scale the data
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

model = LogisticRegression()
model.fit(x_train,y_train)
predictions = model.predict(x_test)
actuals = np.array(y_test)
logistic_acc = model.score(x_test,actuals)
print(logistic_acc)

0.590566037735849


In [354]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

param_grid = [{'solver': ['newton-cg', 'sag', 'lbfgs'],
                 'penalty': ['l2'],
                 'C': [0.1, 0.01, 0.001, 0.5]},
                {'solver': ['saga'],
                 'penalty': ['l1', 'l2'],
                 'C': [0.1, 0.01, 0.001, 0.5]}
                ]

# Use Grid Search to find the best hyperparameters
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5,error_score='raise')
grid_search.fit(x_train, y_train)

# Print the best hyperparameters and corresponding score
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

# Evaluate the model with the best hyperparameters on the testing set
best_lr = grid_search.best_estimator_
y_pred = best_lr.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Best hyperparameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
Best score: 0.5740890688259108
Accuracy: 0.5849056603773585


In [355]:
confusion_matrix(y_test, y_pred)

array([[127, 129],
       [ 91, 183]])

## control

In [356]:
ether_control = pd.concat([ether,df_control], axis=1)
train, test = train_test_split(ether_control, test_size = 0.3)
x_train = train.iloc[0:,1:15]
y_train = train.iloc[0:,0]
x_test = test.iloc[0:,1:15]
y_test = test.iloc[0:,0]
scaler = StandardScaler()
# scale the data
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = LogisticRegression()
model.fit(x_train,y_train)
predictions = model.predict(x_test)
actuals = np.array(y_test)
logistic_acc = model.score(x_test,actuals)
print(logistic_acc)

0.5509433962264151


In [357]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

param_grid = [{'solver': ['newton-cg', 'sag', 'lbfgs'],
                 'penalty': ['l2'],
                 'C': np.logspace(-4, 4, 9)},
                {'solver': ['saga'],
                 'penalty': ['l1', 'l2'],
                 'C': [0.1, 0.01, 0.001, 0.5]}
                ]

# Use Grid Search to find the best hyperparameters
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters and corresponding score
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

# Evaluate the model with the best hyperparameters on the testing set
best_lr = grid_search.best_estimator_
y_pred = best_lr.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Best hyperparameters: {'C': 1000.0, 'penalty': 'l2', 'solver': 'newton-cg'}
Best score: 0.5748987854251012
Accuracy: 0.5490566037735849


In [358]:
confusion_matrix(y_test, y_pred)

array([[130, 117],
       [122, 161]])

# 3. BDM_exlarge

In [359]:
BDM_exlarge_concat = pd.concat([BDM_exlarge,df_na], axis=1)
train, test = train_test_split(BDM_exlarge_concat, test_size = 0.3)
x_train = train.iloc[0:,1:13]
y_train = train.iloc[0:,0]
x_test = test.iloc[0:,1:13]
y_test = test.iloc[0:,0]
scaler = StandardScaler()
# scale the data
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

model = LogisticRegression()
model.fit(x_train,y_train)
predictions = model.predict(x_test)
actuals = np.array(y_test)
logistic_acc = model.score(x_test,actuals)
print(logistic_acc)

0.5924528301886792


In [360]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

param_grid = [{'solver': ['newton-cg', 'sag', 'lbfgs'],
                 'penalty': ['l2'],
                 'C': [0.1, 0.01, 0.001]},
                {'solver': ['saga'],
                 'penalty': ['l1', 'l2'],
                 'C': [0.1, 0.01, 0.001]}
                ]

# Use Grid Search to find the best hyperparameters
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters and corresponding score
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

# Evaluate the model with the best hyperparameters on the testing set
best_lr = grid_search.best_estimator_
y_pred = best_lr.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Best hyperparameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
Best score: 0.5846153846153845
Accuracy: 0.5981132075471698


In [361]:
confusion_matrix(y_test, y_pred)

array([[102, 122],
       [ 91, 215]])

## control

In [362]:
BDM_exlarge_control = pd.concat([BDM_exlarge,df_control], axis=1)
train, test = train_test_split(BDM_exlarge_control, test_size = 0.3)
x_train = train.iloc[0:,1:15]
y_train = train.iloc[0:,0]
x_test = test.iloc[0:,1:15]
y_test = test.iloc[0:,0]
scaler = StandardScaler()
# scale the data
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = LogisticRegression()
model.fit(x_train,y_train)
predictions = model.predict(x_test)
actuals = np.array(y_test)
logistic_acc = model.score(x_test,actuals)
print(logistic_acc)

0.5924528301886792


In [363]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

param_grid = [{'solver': ['newton-cg', 'sag', 'lbfgs'],
                 'penalty': ['l2'],
                 'C': np.logspace(-4, 4, 9)},
                {'solver': ['saga'],
                 'penalty': ['l1', 'l2'],
                 'C': [0.1, 0.01, 0.001, 0.5]}
                ]

# Use Grid Search to find the best hyperparameters
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters and corresponding score
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

# Evaluate the model with the best hyperparameters on the testing set
best_lr = grid_search.best_estimator_
y_pred = best_lr.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Best hyperparameters: {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}
Best score: 0.6040485829959514
Accuracy: 0.5735849056603773


In [364]:
confusion_matrix(y_test, y_pred)

array([[ 78, 164],
       [ 62, 226]])

# 4. BDM

In [365]:
BDM_concat = pd.concat([BDM,df_na], axis=1)
train, test = train_test_split(BDM_concat, test_size = 0.3)
x_train = train.iloc[0:,1:13]
y_train = train.iloc[0:,0]
x_test = test.iloc[0:,1:13]
y_test = test.iloc[0:,0]
scaler = StandardScaler()
# scale the data
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

model = LogisticRegression()
model.fit(x_train,y_train)
predictions = model.predict(x_test)
actuals = np.array(y_test)
logistic_acc = model.score(x_test,actuals)
print(logistic_acc)

0.5792452830188679


In [366]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

param_grid = [{'solver': ['newton-cg', 'sag', 'lbfgs'],
                 'penalty': ['l2'],
                 'C': [0.1, 0.01, 0.001]},
                {'solver': ['saga'],
                 'penalty': ['l1', 'l2'],
                 'C': [0.1, 0.01, 0.001]}
                ]

# Use Grid Search to find the best hyperparameters
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters and corresponding score
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

# Evaluate the model with the best hyperparameters on the testing set
best_lr = grid_search.best_estimator_
y_pred = best_lr.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Best hyperparameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
Best score: 0.6105263157894736
Accuracy: 0.5754716981132075


In [367]:
confusion_matrix(y_test, y_pred)

array([[ 93, 165],
       [ 60, 212]])

## control

In [368]:
BDM_control = pd.concat([BDM,df_control], axis=1)
train, test = train_test_split(BDM_control, test_size = 0.3)
x_train = train.iloc[0:,1:15]
y_train = train.iloc[0:,0]
x_test = test.iloc[0:,1:15]
y_test = test.iloc[0:,0]
scaler = StandardScaler()
# scale the data
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = LogisticRegression()
model.fit(x_train,y_train)
predictions = model.predict(x_test)
actuals = np.array(y_test)
logistic_acc = model.score(x_test,actuals)
print(logistic_acc)

0.590566037735849


In [369]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

param_grid = [{'solver': ['newton-cg', 'sag', 'lbfgs'],
                 'penalty': ['l2'],
                 'C': np.logspace(-4, 4, 9)},
                {'solver': ['saga'],
                 'penalty': ['l1', 'l2'],
                 'C': [0.1, 0.01, 0.001, 0.5]}
                ]

# Use Grid Search to find the best hyperparameters
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters and corresponding score
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

# Evaluate the model with the best hyperparameters on the testing set
best_lr = grid_search.best_estimator_
y_pred = best_lr.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Best hyperparameters: {'C': 10.0, 'penalty': 'l2', 'solver': 'newton-cg'}
Best score: 0.6153846153846154
Accuracy: 0.5886792452830188


In [370]:
confusion_matrix(y_test, y_pred)

array([[100, 156],
       [ 62, 212]])